# Labs Access Analysis: 04 Enrich Data Facebook

## Setup

In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import os
import gc
sns.set()
#sns.set_style("whitegrid") # Seaborn style

/Users/massimo/Documents/GitHub/labs_access_analysis/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Load data, csv
data_csv = pd.read_csv('data/valid_labs.csv')
data_csv['Lat'] = data_csv['Lat'].astype(float)
data_csv['Long'] = data_csv['Long'].astype(float)
data_csv = data_csv[(((~data_csv.Lat.isnull()) & (~data_csv.Long.isnull())))]

In [3]:
data_labs = gpd.GeoDataFrame(
    data_csv,
    crs='EPSG:4326',
    geometry=gpd.points_from_xy(data_csv.Long, data_csv.Lat))

In [4]:
data_labs.head()

,Unnamed: 0,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes,geometry
0,0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN,POINT (2.19434 41.39692)
1,1,MADE Makerspace Barcelona,Fab Lab,Barcelona,BCN,Carrer Noguera Pallaresa 59-61,8014.0,41.373238,2.141084,"Fablabs.io, Hackerspaces.org, Makerspaces.make.co",NaN,NaN,POINT (2.14108 41.37324)
2,2,Soko Tech,Fab Lab,Barcelona,BCN,"Carrer de Vallès i Ribot, 36",8027.0,41.424535,2.188883,Fablabs.io,https://soko.tech/,NaN,POINT (2.18888 41.42454)
3,3,LSCongres,Fab Lab,Barcelona,BCN,"Carrer del Cardenal Tedeschini, 50",8027.0,41.426002,2.182026,Fablabs.io,https://fablabs.io/labs/lscongres,NaN,POINT (2.18203 41.42600)
4,4,Green Fab Lab,Fab Lab,Cerdanyola del Vallès,BCN,"Ctra. BV-1415 (Horta-Cerdanyola), km 7",8290.0,41.450394,2.133441,Fablabs.io,NaN,NaN,POINT (2.13344 41.45039)


In [5]:
data_labs.explore()

In [6]:
isochrones = pd.read_json("data/isochrones.json")

In [7]:
isochrones.head()

,type,bbox,features,metadata,name
0,FeatureCollection,"[2.181814, 41.386685, 2.206919, 41.408001]","[{'type': 'Feature', 'properties': {'group_ind...",{'attribution': 'openrouteservice.org | OpenSt...,Fab Lab Barcelona
1,FeatureCollection,"[2.127469, 41.362337, 2.156052, 41.384602]","[{'type': 'Feature', 'properties': {'group_ind...",{'attribution': 'openrouteservice.org | OpenSt...,MADE Makerspace Barcelona
2,FeatureCollection,"[2.174851, 41.412242, 2.201856, 41.435695]","[{'type': 'Feature', 'properties': {'group_ind...",{'attribution': 'openrouteservice.org | OpenSt...,Soko Tech
3,FeatureCollection,"[2.167014, 41.414324, 2.195139, 41.437322]","[{'type': 'Feature', 'properties': {'group_ind...",{'attribution': 'openrouteservice.org | OpenSt...,LSCongres
4,FeatureCollection,"[2.130189, 41.443331, 2.143093, 41.453097]","[{'type': 'Feature', 'properties': {'group_ind...",{'attribution': 'openrouteservice.org | OpenSt...,Green Fab Lab


## Transform data for geodataframe

In [8]:
isochrones['longitude'] = isochrones['metadata'].apply(lambda x: x['query']['locations'][0][0])
isochrones['latitude'] = isochrones['metadata'].apply(lambda x: x['query']['locations'][0][1])
isochrones['area_units'] = isochrones['metadata'].apply(lambda x: x['query']['area_units'])

In [9]:
isochrones['total_pop'] = isochrones['features'].apply(lambda x: x[0]['properties']['total_pop'])
isochrones['area'] = isochrones['features'].apply(lambda x:x[0]['properties']['area'])
isochrones['reachfactor'] = isochrones['features'].apply(lambda x: x[0]['properties']['reachfactor'])

In [10]:
isochrones['coordinates'] = isochrones['features'].apply(lambda x: x[0]['geometry']['coordinates'])
isochrones['geometry_type'] = isochrones['features'].apply(lambda x: x[0]['geometry']['type'])

In [11]:
isochrones['geometry'] = isochrones['coordinates'].apply(lambda x: Polygon(x[0]))

In [12]:
isochrones = isochrones[['name','longitude','latitude','total_pop','area','reachfactor','geometry']]

In [13]:
isochrones = gpd.GeoDataFrame(
    isochrones,
    geometry=isochrones.geometry,
    crs='EPSG:4326')

In [14]:
isochrones.explore()

In [15]:
isochrones.head()

,name,longitude,latitude,total_pop,area,reachfactor,geometry
0,Fab Lab Barcelona,2.194335,41.396925,84175.0,4351089.58,0.8864,"POLYGON ((2.18181 41.39619, 2.18182 41.39463, ..."
1,MADE Makerspace Barcelona,2.141084,41.373238,88686.0,4388156.04,0.8939,"POLYGON ((2.12747 41.37483, 2.12830 41.37021, ..."
2,Soko Tech,2.188883,41.424535,90167.0,4342469.36,0.8846,"POLYGON ((2.17485 41.42471, 2.17489 41.42377, ..."
3,LSCongres,2.182026,41.426002,95371.0,4470900.19,0.9108,"POLYGON ((2.16701 41.42773, 2.16735 41.42654, ..."
4,Green Fab Lab,2.133441,41.450394,29.0,668382.48,0.1362,"POLYGON ((2.13019 41.44688, 2.13044 41.44663, ..."


In [16]:
isochrones.to_file("data/lab_isochrones.geojson",driver='GeoJSON')

In [17]:
columns = {
    'name': 'Name',
}
isochrones.rename(columns=columns, inplace=True)

In [18]:
labs = isochrones.sjoin(data_labs, how='inner')
labs.head()

,Name_left,longitude,latitude,total_pop,area,reachfactor,geometry,index_right,Unnamed: 0,Name_right,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes
0,Fab Lab Barcelona,2.194335,41.396925,84175.0,4351089.58,0.8864,"POLYGON ((2.18181 41.39619, 2.18182 41.39463, ...",0,0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
37,ATTA33,2.189540,41.400318,94386.0,4680951.15,0.9536,"POLYGON ((2.17617 41.40010, 2.17745 41.39469, ...",0,0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
0,Fab Lab Barcelona,2.194335,41.396925,84175.0,4351089.58,0.8864,"POLYGON ((2.18181 41.39619, 2.18182 41.39463, ...",37,43,ATTA33,Service,Barcelona,BCN,"Carrer de Tànger, 33",8018.0,41.400318,2.189540,Manually added,https://atta33.com/,NaN
21,AKASHA Hub Barcelona,2.191146,41.411054,95436.0,4597828.76,0.9367,"POLYGON ((2.17899 41.41306, 2.17919 41.40872, ...",37,43,ATTA33,Service,Barcelona,BCN,"Carrer de Tànger, 33",8018.0,41.400318,2.189540,Manually added,https://atta33.com/,NaN
37,ATTA33,2.189540,41.400318,94386.0,4680951.15,0.9536,"POLYGON ((2.17617 41.40010, 2.17745 41.39469, ...",37,43,ATTA33,Service,Barcelona,BCN,"Carrer de Tànger, 33",8018.0,41.400318,2.189540,Manually added,https://atta33.com/,NaN


In [19]:
labs.shape

(104, 20)

In [20]:
labs.explore()

In [21]:
del labs['Lat']
del labs['Long']

In [22]:
columns = {
    'Name': 'name',
    'Type': 'type',
    'City': 'city',
    'Cluster':  'cluster',
    'Address': 'address',
    'CAP': 'zipcode',
    'Source': 'source',
    'URL': 'url',
    'Notes': 'notes',
    'geomtry_y': 'geomtry'
}
labs.rename(columns=columns, inplace=True)

In [23]:
labs.rename(columns=columns, inplace=True)

In [24]:
labs.to_file("data/labs_isochrones.geojson",driver='GeoJSON')

In [25]:
labs.tail()

,Name_left,longitude,latitude,total_pop,area,reachfactor,geometry,index_right,Unnamed: 0,Name_right,type,city,cluster,address,zipcode,source,url,notes
48,Locanda Officina Monumentale,9.173905,45.489169,33427.0,2511071.05,0.5116,"POLYGON ((9.15953 45.48735, 9.16029 45.48512, ...",48,54,Locanda Officina Monumentale,Incubator/Accelerator,Milan,MI,Via Galileo Ferraris 1,20154.0,Manually added,https://www.lommilano.it/,NaN
44,BDN Lab,2.238115,41.442801,56797.0,2947604.66,0.6005,"POLYGON ((2.22578 41.44674, 2.22604 41.44412, ...",44,50,BDN Lab,Fab Lab,Badalona,BCN,"Carrer del General Weyler, 128",8912.0,Manually added,https://bdnlab.org/,NaN
47,DFactory Barcelona,2.138019,41.336487,20821.0,1016551.68,0.2071,"POLYGON ((2.12539 41.33739, 2.12747 41.33284, ...",47,53,DFactory Barcelona,Incubator/Accelerator,Barcelona,BCN,"Carrer 27, 10-16 Sector BZ Zona Franca",8040.0,Manually added,https://www.dfactorybcn.org/,NaN
50,Share Makerspace,9.177185,45.560405,11593.0,2319424.78,0.4725,"POLYGON ((9.16543 45.56093, 9.16559 45.56076, ...",50,56,Share Makerspace,Makerspace,Cusano Milanino,MI,"Via Giuseppe Zucchi, 39/G/Interno 20",20095.0,Manually added,https://www.sharemakers.it/,NaN
52,Camins Makers,2.111370,41.388787,61277.0,3547085.01,0.7226,"POLYGON ((2.10075 41.38836, 2.10219 41.38307, ...",52,58,Camins Makers,University Lab,Barcelona,BCN,​C/ Jordi Girona 1-3,8034.0,Manually added,https://caminsmakers.upc.edu/,NaN


# Load facebook data

Spain

[https://data.humdata.org/dataset/spain-high-resolution-population-density-maps-demographic-estimates](https://data.humdata.org/dataset/spain-high-resolution-population-density-maps-demographic-estimates)

Italy

[https://data.humdata.org/dataset/italy-high-resolution-population-density-maps-demographic-estimates](https://data.humdata.org/dataset/italy-high-resolution-population-density-maps-demographic-estimates)

[population_ita_2019-07-01.csv.zip](https://data.humdata.org/dataset/0eb77b21-06be-42c8-9245-2edaff79952f/resource/1e96f272-7d86-4108-b4ca-5a951a8b11a0/download/population_ita_2019-07-01.csv.zip)

[ITA_youth_15_24.csv.zip](https://data.humdata.org/dataset/0eb77b21-06be-42c8-9245-2edaff79952f/resource/46080f74-58bc-4d02-b761-56bb7d42076f/download/ita_youth_15_24_2019-08-03.csv.zip)



In [26]:
labs_ita = labs[labs.cluster == "MI"]
labs_esp = labs[labs.cluster == "BCN"]

In [27]:
url_ita_15_24 = "https://data.humdata.org/dataset/0eb77b21-06be-42c8-9245-2edaff79952f/resource/46080f74-58bc-4d02-b761-56bb7d42076f/download/ita_youth_15_24_2019-08-03.csv.zip"
url_ita_population = "https://data.humdata.org/dataset/0eb77b21-06be-42c8-9245-2edaff79952f/resource/1e96f272-7d86-4108-b4ca-5a951a8b11a0/download/population_ita_2019-07-01.csv.zip"
url_ita_men = "https://data.humdata.org/dataset/0eb77b21-06be-42c8-9245-2edaff79952f/resource/03d5be33-57b8-4c02-879e-0b458dbdd554/download/ita_men_2019-08-03.csv.zip"
url_ita_women = "https://data.humdata.org/dataset/0eb77b21-06be-42c8-9245-2edaff79952f/resource/b761d291-4ad5-47cb-a1eb-6962065918d3/download/ita_women_2019-08-03.csv.zip"

In [ ]:
ita_population.head(3)

In [ ]:
def getgeodataframearea(isochrones, points,x='longitude',y='latitude'):
    env = env = isochrones.envelope.unary_union.bounds
    area = points[(points[x] >= env[0]) & (points[x] <= env[2]) & (points[y] >= env[1]) & (points[y] <= env[3])]
    geodataframe = gpd.GeoDataFrame(
        points,
        crs='EPSG:4326',
        geometry=gpd.points_from_xy(points[x], points[y]))
    return geodataframe

In [29]:
def adddataurl(data, url, column):
    new_data = pd.read_csv(url)
    points_cluster_population = getgeodataframearea(data, new_data,x='Lon',y='Lat')
    # save memory - destroy dataframe unused
    # del [[isochrones,ita_population,ita_15_24]]
    del [[isochrones,new_data]]
    gc.collect()
    isochrones=pd.DataFrame()
    new_data=pd.DataFrame()
    sjoin_lab_points = points_cluster_population.sjoin(data, how='inner')
    data[column] = data.name.apply(lambda x: round(sjoin_lab_points[sjoin_lab_points.name == x].Population.sum()))
    return data

In [31]:
adddataurl(labs_ita, url_ita_population, 'population_fb_ita')

NameError: name 'getgeodataframearea' is not defined

In [ ]:
labs_ita.head()

In [ ]:
# Configure font
matplotlib.rcParams['font.sans-serif'] = "Fira Sans"
matplotlib.rcParams['font.family'] = "sans-serif"
sns.set_context("poster")
# Plot the data
labs_ita2 = labs_ita.set_index("name")
labs_ita2.sort_values(by=["population_fb_ita"])["population_fb_ita"].plot(kind="barh", figsize=(20,12))
plt.xlabel('Population reached')
plt.ylabel('Labs')
plt.savefig("svg/labs_ita_fbpop.svg", bbox_inches="tight")
plt.savefig("png/labs_ita_fbpop.png", dpi=300, bbox_inches="tight")